# Differentiating CVQNN layers with tensorflow

In Piquasso, one can easily create CVQNN circuits using the `cvqnn` module. In these circuits, the layers are constructed according to [Continuous-variable quantum neural networks](https://arxiv.org/abs/1806.06871).

In [1]:
import piquasso as pq

d = 3 # Number of qumodes

layer_count = 5

weights = pq.cvqnn.generate_random_cvqnn_weights(layer_count=layer_count, d=d)

cvqnn_layers = pq.cvqnn.create_layers(weights)

cvqnn_layers.instructions

2024-02-15 23:36:52.797301: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 23:36:52.840692: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 23:36:52.841602: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-15 23:36:53.617271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[<pq.Beamsplitter(theta=0.15268730562725547, phi=0.015520974280387095, modes=(0, 1))>,
 <pq.Beamsplitter(theta=0.03460811747147755, phi=0.16064727298957726, modes=(1, 2))>,
 <pq.Beamsplitter(theta=0.10756435256811797, phi=0.20370822264343708, modes=(0, 1))>,
 <pq.Phaseshifter(phi=0.01764887005536274, modes=(0,))>,
 <pq.Phaseshifter(phi=0.18180832204998942, modes=(1,))>,
 <pq.Squeezing(r=-0.013229437324357668, phi=0.0, modes=(0,))>,
 <pq.Squeezing(r=0.0028532552979723285, phi=0.0, modes=(1,))>,
 <pq.Squeezing(r=0.0032642310899366938, phi=0.0, modes=(2,))>,
 <pq.Beamsplitter(theta=-0.03858950915234623, phi=-0.0859130379199507, modes=(0, 1))>,
 <pq.Beamsplitter(theta=-0.11074885597219278, phi=-0.015631283953091265, modes=(1, 2))>,
 <pq.Beamsplitter(theta=0.027088618789303395, phi=0.007158329838990549, modes=(0, 1))>,
 <pq.Phaseshifter(phi=-0.0769606896158746, modes=(0,))>,
 <pq.Phaseshifter(phi=0.07460621145749698, modes=(1,))>,
 <pq.Displacement(r=0.006770145968560146, phi=-0.05809737775

Piquasso automatically sets up a subprogram containing the instructions of the desired CVQNN layer. Now we can embed this subprogram in any program. Let's choose the input state as a pure displaced state.

In [2]:
with pq.Program() as program:
    pq.Q() | pq.Vacuum()

    for i in range(d):
        pq.Q(i) | pq.Displacement(r=0.1)

    pq.Q() | cvqnn_layers

Now, we need to choose the simulator which executes the instructions. Since a CVQNN layer includes non-linear terms, we definitely need to perform the simulation in Fock space. Since our initial state is pure, we can use `PureFockSimulator`.

In [3]:
cutoff = 10

simulator = pq.PureFockSimulator(d, pq.Config(cutoff=cutoff))

final_state = simulator.execute(program).state

After obtaining the state, we can calculate several things, e.g. the expectation value of the position operator on mode 0.

In [4]:
print("Mean position on mode 0:")
final_state.mean_position(mode=0)

Mean position on mode 0:


0.09641507191192268

In order to differentiate it, we need to modify the simulation. In order to do that, we can use `TensorflowCalculator`, which replaces NumPy to TensorFlow under the hood. Then, by using `tensorflow.GradientTape`, we can calculate the gradient of the expectation value of the position operator on mode 0.

In [5]:
import tensorflow as tf

simulator = pq.PureFockSimulator(
    d, pq.Config(cutoff=cutoff), calculator=pq.TensorflowCalculator()
)

weights = tf.Variable(weights)

with tf.GradientTape() as tape:
    cvqnn_layers = pq.cvqnn.create_layers(weights)

    with pq.Program() as program:
        pq.Q() | pq.Vacuum()

        for i in range(d):
            pq.Q(i) | pq.Displacement(r=0.1)

        pq.Q() | cvqnn_layers

    simulator.execute(program)

    final_state = simulator.execute(program).state

    mean_position = final_state.mean_position(0)

grad_mean_position = tape.gradient(mean_position, weights)

2024-02-15 23:36:57.600495: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


However, one can also use `tf.function` for this task. Refactoring everything into a function, we can use the `tf.function` decorator.

In [6]:
def calculate_mean_position(weights):
    d = pq.cvqnn.get_number_of_modes(weights.shape[1])

    simulator = pq.PureFockSimulator(
        d,
        pq.Config(cutoff=cutoff, normalize=False),
        calculator=pq.TensorflowCalculator(),
    )

    with tf.GradientTape() as tape:
        cvqnn_layers = pq.cvqnn.create_layers(weights)

        preparation = [pq.Vacuum()] + [
            pq.Displacement(r=0.1).on_modes(i) for i in range(d)
        ]

        program = pq.Program(instructions=preparation + cvqnn_layers.instructions)

        simulator.execute(program)

        final_state = simulator.execute(program).state

        mean_position = final_state.mean_position(0)

    return mean_position, tape.gradient(mean_position, weights)

enhanced_calculate_mean_position = tf.function(jit_compile=True)(calculate_mean_position)

enhanced_calculate_mean_position(weights)

2024-02-15 23:43:07.199526: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xc25c6a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-02-15 23:43:07.199555: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-02-15 23:44:11.662779: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


The compilation takes some time, but after it has been compiled, a significant speedup is observed. We can also compare the runtimes of the compiled and non-compiled function.

In [ ]:
%timeit calculate_mean_position(weights)
%timeit enhanced_calculate_mean_position(weights)

1.08 s ± 45.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


KeyboardInterrupt: 